In [4]:
from parsemrz import parse
from imagetomrz import ExtractData

In [5]:
ExtractData('C:\\Users\\iyedn\\OneDrive\\Desktop\\stage\\mrz-detection-master\\img\\210316052514-iceland-gender-neutral-passport.jpg')
data = ExtractData('C:\\Users\\iyedn\\OneDrive\\Desktop\\stage\\mrz-detection-master\\img\\210316052514-iceland-gender-neutral-passport.jpg')

In [6]:
parse(data)

THURIDUR OESP


['THURIDUR OESP',
 'AEVARSDOTTIR',
 'P',
 'A',
 'ISL',
 'A3536444',
 'ISL',
 '121212',
 'unspecified',
 '310310',
 '121212<1239',
 '7',
 '3',
 '8',
 '6',
 '8']